In [ ]:
import pandas as pd

df = pd.read_csv('assets/data/datos.csv', delimiter=';')

#print(df.head())

print(df.columns)

Index(['Categoria', 'Enlace', 'Nombre', 'ImagenURL', 'Ingredientes',
       'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
       'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
       'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
       'Potasio(mg)'],
      dtype='object')


In [10]:
import os
import pandas as pd
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder

# Cargar variables de entorno
load_dotenv()

# Inicializar Azure OpenAI Embeddings
embeddings_model = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
    #api_version=os.getenv("api_version_openai_emb"),
    #deployment_name= os.getenv("deploy_name_emb")
)

# Cargar el dataset
df = pd.read_csv('assets/data/datos.csv', delimiter=';')  # Asegúrate de reemplazar con tu ruta

# Columnas de texto que queremos convertir
columnas_texto = ['Categoria', 'Nombre', 'Ingredientes']

# Aplicar embeddings a las columnas de texto y convertir a números
for col in columnas_texto:
    df[col + "_emb"] = df[col].astype(str).apply(lambda x: sum(embeddings_model.embed_query(x)))

# Codificar las categorías con números (Label Encoding)
label_encoder = LabelEncoder()
df["Categoria_Num"] = label_encoder.fit_transform(df["Categoria"].astype(str))

# Columnas numéricas originales + nuevas columnas transformadas
columnas_numericas = [
    'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
    'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
    'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
    'Potasio(mg)', 'Categoria_Num', 'Nombre_emb', 'Ingredientes_emb'
]

# Reemplazar NaN con 0
df[columnas_numericas] = df[columnas_numericas].fillna(0)

# Crear la nueva columna con la suma total
df["Suma_Total"] = df[columnas_numericas].sum(axis=1)

# Guardar el dataset con la nueva columna
df.to_csv("dataset_numerico.csv", index=False)

print("Dataset convertido y guardado en dataset_numerico.csv ✅")


Dataset convertido y guardado en dataset_numerico.csv ✅


In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

load_dotenv()

def similitud_cosenos(vect1: list, vect2: list) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity


df = pd.read_csv('assets/data/datos.csv', delimiter=";") 

#respuestas = df["Ingredientes"].dropna().tolist()

respuestas = df[["Ingredientes", "Nombre", "Categoria"]].dropna().values.tolist()

embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

query = [embeddings.embed_query(text[1]) for text in respuestas]


In [5]:

query_client = "Receta sin arroz"
query_embeddings = embeddings.embed_query(query_client)


response_option = [similitud_cosenos(query_embeddings, q) for q in query]

pos = np.argmax(response_option)
request = respuestas[pos]

print(f"{query_client} - rta: {request}")

Receta sin arroz - rta: [' | 1 libra de carne molida de res magra (15% de grasa) | 1 cebolla mediana, picada | 1 chile pimiento pequeño o jalapeño, picado | 1 taza de arroz, sin cocer | 2 tazas de tomate picado, fresco o enlatado, en su jugo | 2 tazas de agua | 1 cucharada de chile en polvo (o más, a su gusto) | 1 cucharada de orégano deshidratado | 1 cucharadita de sal | 1/2 taza (2 onzas) de queso cheddar, rallado', 'Cazuela de arroz picante', 'Cena']


In [ ]:

query_client = "Receta sin arroz con 500 calorias"
query_embeddings = embeddings.embed_query(query_client)


response_option = [similitud_cosenos(query_embeddings, q) for q in query]


pos = np.argmax(response_option)
request_ingredientes, request_nombre, request_categoria = respuestas[pos]

print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")

Receta sin arroz
Receta: Cazuela de arroz picante
Categoría: Cena
Ingredientes:  | 1 libra de carne molida de res magra (15% de grasa) | 1 cebolla mediana, picada | 1 chile pimiento pequeño o jalapeño, picado | 1 taza de arroz, sin cocer | 2 tazas de tomate picado, fresco o enlatado, en su jugo | 2 tazas de agua | 1 cucharada de chile en polvo (o más, a su gusto) | 1 cucharada de orégano deshidratado | 1 cucharadita de sal | 1/2 taza (2 onzas) de queso cheddar, rallado


In [3]:
from langchain_openai import AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

load_dotenv()

def similitud_cosenos(vect1: list, vect2: list) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity

# Cargar dataset original
df = pd.read_csv('assets/data/datos.csv', delimiter=";") 

# Extraer información relevante para embeddings
columnas_relevantes = ["Categoria", "Nombre", "Ingredientes", "Calorias"]

df_relevante = df[columnas_relevantes].dropna()

# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Crear embeddings para todas las columnas relevantes
for columna in columnas_relevantes:
    print(f"Convirtiendo columna a embeddings: {columna}")
    df_relevante[f"{columna}_Embeddings"] = df_relevante[columna].apply(lambda x: print(f"Procesando: {x}") or embeddings.embed_query(str(x)))

# Guardar nuevo dataset con embeddings
df_relevante.to_csv('assets/data/datos_embeddings.csv', index=False, sep=";")

print("Dataset con embeddings creado exitosamente.")


Convirtiendo columna a embeddings: Categoria
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Desayuno
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando: Almuerzo
Procesando

In [9]:
import numpy as np
import pandas as pd
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

def similitud_cosenos(vect1: np.ndarray, vect2: np.ndarray) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    return dot_product / (norm_A * norm_B)

# Cargar dataset con embeddings
df_embeddings = pd.read_csv('assets/data/datos_embeddings.csv', delimiter=";")

for columna in ["Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings"]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").replace(",", "").split()])
    )


for columna in ["Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings"]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").split()]) if isinstance(x, str) else x
    )


# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Consulta del usuario
query_client = "Receta con platano"
query_embeddings = np.array(embeddings.embed_query(query_client))

# Calcular similitud con todas las filas del dataset (usando los embeddings combinados)
response_option = [
    similitud_cosenos(query_embeddings, np.mean(np.vstack([
        row["Categoria_Embeddings"], 
        row["Nombre_Embeddings"], 
        row["Ingredientes_Embeddings"], 
        row["Calorias_Embeddings"]
    ]), axis=0))  
    for _, row in df_embeddings.iterrows()
]

# Encontrar la receta más similar
pos = np.argmax(response_option)
request_ingredientes = df_embeddings.loc[pos, "Ingredientes"]
request_nombre = df_embeddings.loc[pos, "Nombre"]
request_categoria = df_embeddings.loc[pos, "Categoria"]

# Imprimir resultados
print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")



Receta con platano
Receta: Galletas de plátano y avena
Categoría: Bajo en grasa
Ingredientes:  | 2 plátanos (bananas) bien maduros | 1 taza de avena (instantánea o tradicional) | 1/2 cucharadita de canela molida | 1/2 cucharadita de vainilla | 1/4 taza de pasas


In [ ]:
import numpy as np
import pandas as pd
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
import json

load_dotenv()

def similitud_cosenos(vect1: np.ndarray, vect2: np.ndarray) -> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    return dot_product / (norm_A * norm_B)

# Cargar dataset con embeddings
df_embeddings = pd.read_csv('assets/data/datos_embeddings.csv', delimiter=";")

for columna in ["Categoria_Embeddings", "Nombre_Embeddings", "Ingredientes_Embeddings", "Calorias_Embeddings"]:
    df_embeddings[columna] = df_embeddings[columna].apply(
        lambda x: np.array([float(i) for i in x.strip("[]").replace(",", "").split()])
    )

# Configurar embeddings
embeddings = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
)

# Función para filtrar recetas según los ingredientes deseados y no deseados, y las calorías
def filtrar_recetas(json_query, df):
    calorias_deseadas = json_query.get("calorías", None)
    ingredientes_deseados = json_query.get("ingredientes_deseados", [])
    ingredientes_no_deseados = json_query.get("ingredientes_no_deseados", [])

    # Filtrar recetas por calorías si se especifica
    if calorias_deseadas:
        calorias_deseadas_value = float(calorias_deseadas.replace("kcal", "").strip())
        df = df[df["Calorias"].apply(lambda x: float(str(x).replace("kcal", "").strip()) <= calorias_deseadas_value)]

    # Filtrar recetas por ingredientes
    if ingredientes_deseados:
        df = df[df["Ingredientes"].apply(lambda x: all(ingrediente in x.lower() for ingrediente in ingredientes_deseados))]

    if ingredientes_no_deseados:
        df = df[df["Ingredientes"].apply(lambda x: all(ingrediente not in x.lower() for ingrediente in ingredientes_no_deseados))]

    return df

# Ejemplo de query en formato JSON
query_client = {
    "calorías": "500kcal",
    "ingredientes_deseados": ["arroz"],
    "ingredientes_no_deseados":  ["pollo"]
}

# Filtrar recetas usando los criterios del JSON
df_filtrado = filtrar_recetas(query_client, df_embeddings)

# Convertir el diccionario a una cadena de texto
query_text = f"Calorías: {query_client['calorías']}, Ingredientes deseados: {', '.join(query_client['ingredientes_deseados'])}, Ingredientes no deseados: {', '.join(query_client['ingredientes_no_deseados'])}"

# Pasar la cadena a la función de embeddings
query_embeddings = np.array(embeddings.embed_query(query_text))

# Calcular similitud con todas las filas del dataset filtrado
response_option = [
    similitud_cosenos(query_embeddings, np.mean(np.vstack([
        row["Categoria_Embeddings"], 
        row["Nombre_Embeddings"], 
        row["Ingredientes_Embeddings"], 
        row["Calorias_Embeddings"]
    ]), axis=0))  
    for _, row in df_filtrado.iterrows()
]

# Verificar si la lista de similitudes tiene elementos
if len(response_option) > 0:
    pos = np.argmax(response_option)  # Obtener la posición del máximo valor
    request_ingredientes = df_filtrado.loc[pos, "Ingredientes"]
    request_nombre = df_filtrado.loc[pos, "Nombre"]
    request_categoria = df_filtrado.loc[pos, "Categoria"]
    
    # Imprimir resultados
    print(f"{query_client}\nReceta: {request_nombre}\nCategoría: {request_categoria}\nIngredientes: {request_ingredientes}")
else:
    print("No se encontraron opciones de respuesta.")


KeyError: np.int64(7)